In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from modeling.model_trip_GRU_make3_Pre_Parellel import *
from modeling.evaluation_trip_Pre_Parellel import *
from sklearn.preprocessing import LabelEncoder

In [2]:
data_spot = pd.read_excel('D:/2019.01.04~여행앱개발/modeling/data_spot_5_fold.xlsx')

In [3]:
new_id = pd.DataFrame(columns=["Spot","new_id"])
new_id["Spot"] = data_spot.Spot.unique()
new_id["new_id"] = range(data_spot.Spot.nunique())

In [4]:
spot = pd.read_excel('../최종DB관리/Spot_VSS/2019.02.13.최종_통합DB2.xlsx')

In [5]:
new_id_spot = new_id.merge(spot,on="Spot",how="left")

In [6]:
data_spot.groupby('Tag_encoder').size()

Tag_encoder
0    13664
1     1063
2     5538
3     2375
4     5562
5      146
dtype: int64

In [26]:
train = data_spot[data_spot["k"]!=0]
test = data_spot[data_spot["k"]==0]

In [27]:
class args():
    is_training = True
    retraining = True
    layers = 1
    rnn_size = 50 # , 20, 5 ,50
    n_epochs = 10
    batch_size = 50
    dropout_p_hidden= 1.0
    learning_rate = 0.00001
    decay = 0.96
    decay_steps = 5e3
    initializer = 'xavier'
    init_as_normal = False
    reset_after_session = True
    session_key = 'Session_id'
    attribute_key = 'Tag_encoder'
    item_key = 'Spot'
    time_key = 'Session_num'
    grad_cap = 0
    attribute = new_id_spot.Tag_encoder.nunique() 
    max_ckpt_keep = 105
    test_model = 6
    sigma=0
    checkpoint_dir = 'Tag_encoder_0' 
    loss = 'top1'
    final_act = 'linear'
    hidden_act = 'tanh'
    item_table = new_id_spot

In [28]:
tf.reset_default_graph()

In [29]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
arg = args()
models = GRU4Rec(sess,arg)

Checkpoint Dir not found I make it
(50, 50)


In [30]:
models.fit(train)

fitting model....
Epoch 0	Step 1	lr: 0.000010	loss: 0.989986
Epoch: 0 cost: 0.9899319410324097
Epoch: 1 cost: 0.9896277785301208
Epoch: 2 cost: 0.9887274503707886
Epoch: 3 cost: 0.9869486093521118
Epoch: 4 cost: 0.9847249388694763
Epoch: 5 cost: 0.9831668138504028
Epoch: 6 cost: 0.9822213053703308
Epoch: 7 cost: 0.981715202331543
Epoch: 8 cost: 0.9814015030860901
Epoch: 9 cost: 0.9814700484275818


In [33]:
arg = args()
arg.is_training = False
for s in range(5):
    print("k :",s)
    for e in range(0,10):    
        print("Epoch:",e)
        tf.reset_default_graph()
        arg.ckechpoint_dir = "Tag_encoder_"+str(s)
        arg.test_model = e
        config = tf.ConfigProto()
        config.gpu_options.allow_growth=True
        sess = tf.Session(config=config)
        models = GRU4Rec(sess,arg)
        train = data_spot[data_spot["k"]!=s]
        test = data_spot[data_spot["k"]==s]

        for i in [5,10,15,20]:
            l_old=evaluate_sessions_batch(models,test,cut_off=i, session_key = 'Session_id', 
                                          time_key= 'Session_num',batch_size=50)
            print("top :",i,l_old)

k : 0
Epoch: 0
(50, 50)
INFO:tensorflow:Restoring parameters from Tag_encoder_0/attribute_model_Tag_encoder-0


D:\2019.01.04~여행앱개발\TripRecommendation\modeling\evaluation_trip_Pre_Parellel.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test_data.sort_values([session_key, time_key], inplace=True)


top : 5 (0.7034700315457413, 0.5887136347704174)
top : 10 (1.0, 0.6406063792499124)
top : 15 (1.0, 0.6406063792499124)
top : 20 (1.0, 0.6406063792499124)
Epoch: 1
(50, 50)
INFO:tensorflow:Restoring parameters from Tag_encoder_0/attribute_model_Tag_encoder-1
top : 5 (0.7070452155625657, 0.5915352260778128)
top : 10 (1.0, 0.6429512793550648)
top : 15 (1.0, 0.6429512793550648)
top : 20 (1.0, 0.6429512793550648)
Epoch: 2
(50, 50)
INFO:tensorflow:Restoring parameters from Tag_encoder_0/attribute_model_Tag_encoder-2
top : 5 (0.7158780231335437, 0.5943217665615141)
top : 10 (1.0, 0.6448370136698212)
top : 15 (1.0, 0.6448370136698212)
top : 20 (1.0, 0.6448370136698212)
Epoch: 3
(50, 50)
INFO:tensorflow:Restoring parameters from Tag_encoder_0/attribute_model_Tag_encoder-3
top : 5 (0.7200841219768664, 0.5972660357518403)
top : 10 (1.0, 0.6466246056782338)
top : 15 (1.0, 0.6466246056782338)
top : 20 (1.0, 0.6466246056782338)
Epoch: 4
(50, 50)
INFO:tensorflow:Restoring parameters from Tag_encoder_

top : 5 (0.6939901477832512, 0.5756978653530377)
top : 10 (1.0, 0.6296518883415435)
top : 15 (1.0, 0.6296518883415435)
top : 20 (1.0, 0.6296518883415435)
Epoch: 3
(50, 50)
INFO:tensorflow:Restoring parameters from Tag_encoder_0/attribute_model_Tag_encoder-3
top : 5 (0.6945812807881774, 0.5763218390804598)
top : 10 (1.0, 0.6300098522167483)
top : 15 (1.0, 0.6300098522167483)
top : 20 (1.0, 0.6300098522167483)
Epoch: 4
(50, 50)
INFO:tensorflow:Restoring parameters from Tag_encoder_0/attribute_model_Tag_encoder-4
top : 5 (0.6935960591133005, 0.5730870279146142)
top : 10 (1.0, 0.6248308702791459)
top : 15 (1.0, 0.6248308702791459)
top : 20 (1.0, 0.6248308702791459)
Epoch: 5
(50, 50)
INFO:tensorflow:Restoring parameters from Tag_encoder_0/attribute_model_Tag_encoder-5
top : 5 (0.6916256157635468, 0.570197044334975)
top : 10 (1.0, 0.6213990147783248)
top : 15 (1.0, 0.6213990147783248)
top : 20 (1.0, 0.6213990147783248)
Epoch: 6
(50, 50)
INFO:tensorflow:Restoring parameters from Tag_encoder_0